In [2]:
!pip install tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 MB 1.3 MB/s eta 0:00:0000:0100:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 1.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 1.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 623.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181.8 kB 704.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 1.5 MB/s

In [4]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, losses, Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [6]:

# Load the data
data_c7_1 = pd.read_csv("C7-1.csv")
data_c7_2 = pd.read_csv("C7-2.csv")

# Drop the "Timestamp" column
data_c7_1 = data_c7_1.drop(columns=["Timestamp"])
data_c7_2 = data_c7_2.drop(columns=["Timestamp"])

# Normalize the data
scaler = MinMaxScaler()
data_c7_1_normalized = scaler.fit_transform(data_c7_1)
data_c7_2_normalized = scaler.transform(data_c7_2)

# Split the data from C7-1.csv into training and validation sets
x_train, x_val = train_test_split(data_c7_1_normalized, test_size=0.2, random_state=42)


In [7]:

# Define the autoencoder
latent_dim = 10  # Adjust as needed

class AnomalyAutoencoder(Model):
    def __init__(self, latent_dim):
        super(AnomalyAutoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            layers.Dense(128, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(latent_dim, activation='relu')
        ])
        
        self.decoder = tf.keras.Sequential([
            layers.Dense(64, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(25, activation='sigmoid')  # 25 columns in the dataset
        ])
        
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Create and compile the autoencoder
autoencoder = AnomalyAutoencoder(latent_dim)
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())


In [8]:

# Train the autoencoder
autoencoder.fit(x_train, x_train, epochs=50, batch_size=32, validation_data=(x_val, x_val))


Epoch 1/50
425/425 [==============================] - 1s 826us/step - loss: 0.0109 - val_loss: 0.0021
Epoch 2/50
425/425 [==============================] - 0s 701us/step - loss: 0.0013 - val_loss: 9.1721e-04
Epoch 3/50
425/425 [==============================] - 0s 697us/step - loss: 6.6289e-04 - val_loss: 7.1766e-04
Epoch 4/50
425/425 [==============================] - 0s 700us/step - loss: 5.3642e-04 - val_loss: 6.1773e-04
Epoch 5/50
425/425 [==============================] - 0s 702us/step - loss: 4.6041e-04 - val_loss: 5.5581e-04
Epoch 6/50
425/425 [==============================] - 0s 701us/step - loss: 4.1372e-04 - val_loss: 4.6467e-04
Epoch 7/50
425/425 [==============================] - 0s 700us/step - loss: 3.7399e-04 - val_loss: 4.3900e-04
Epoch 8/50
425/425 [==============================] - 0s 690us/step - loss: 3.4302e-04 - val_loss: 4.3750e-04
Epoch 9/50
425/425 [==============================] - 0s 691us/step - loss: 3.1920e-04 - val_loss: 4.0370e-04
Epoch 10/50
425/425 [=

In [9]:

# Predict on the data from C7-2.csv
reconstructions = autoencoder.predict(data_c7_2_normalized)

# Calculate the Mean Squared Error for each reconstruction
mse = np.mean(np.power(data_c7_2_normalized - reconstructions, 2), axis=1)

# Set a threshold for anomaly detection (e.g., 95th percentile of the MSE on the training data)
threshold = np.percentile(mse, 95)

# Classify as normal (0) or anomaly (1)
anomalies = np.where(mse > threshold, 1, 0)


1085/1085 [==============================] - 0s 264us/step


In [13]:
autoencoder.save("anomaly_detection_model_dir")

INFO:tensorflow:Assets written to: anomaly_detection_model_dir/assets


INFO:tensorflow:Assets written to: anomaly_detection_model_dir/assets


In [15]:
loaded_model = tf.keras.models.load_model("anomaly_detection_model_dir")


In [23]:
autoencoder.build(input_shape=(None, 25))  # 25 is the number of features in your dataset
autoencoder.save_weights("anomaly_detection_weights.h5")


In [24]:
import os
print(os.getcwd())


/Users/ted


In [26]:
directory_path = "/Users/ted/my_models/"
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
autoencoder.save_weights(directory_path + "anomaly_detection_weights.h5")


In [27]:
autoencoder.save_weights("anomaly_detection_weights.h5")


In [28]:
import subprocess

# Path to the file or directory you want to open
path_to_open = "/Users/ted/anomaly_detection_weights.h5"

# Open the path in Finder
subprocess.run(["open", "-R", path_to_open])


CompletedProcess(args=['open', '-R', '/Users/ted/anomaly_detection_weights.h5'], returncode=0)